# Linear Regression from scratch

## 1.Libraries

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from copy import deepcopy

## 2. Linear Regression with Normal Equation

### Implementation

In [2]:
class LinearRegression:
    @staticmethod
    def add_bias(X):
        bias = np.ones((X.shape[0],1))
        X = X.values
        X = np.concatenate([bias, X], axis=1)
        return X
    
    def pre_process(self):
        self.X = self.add_bias(self.X)
        y = self.y.values
        self.y = y.reshape((-1, 1))
    
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.pre_process()
        self.w = np.zeros((1,X.shape[1]))
        
    def predict(self, X):
        X = self.add_bias(X)
        y = np.dot(X,self.w)
        return y
        
    
    def fit(self):
        X, X_t = self.X, self.X.T
        psudo_inv = np.linalg.inv(np.dot(X_t,X))
        self.w = np.dot(np.dot(psudo_inv,X_t),self.y)

### 2.2 Optimal Parameters 

$$
w = (X^TX)^{-1}X^TY
$$

### 2.3 Cost Function

In [228]:
def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred)**2)

$$
MSE = \frac{1}{N} \sum_{i=1}^{N} (y_i - \hat{y_i})^2
$$

## 3. Train the Model

### 3.1 Split data

In [4]:
train = pd.read_csv('Dataset/Problem 1/train_set.csv')
test = pd.read_csv('Dataset/Problem 1/test_set.csv')

In [5]:
train.head()

,x,target
0,2.391265,0.692274
1,4.214828,4.078137
2,2.801953,2.614283
3,2.448590,1.465649
4,4.671812,-1.863850


In [6]:
X_train, X_test = train[['x']], test[['x']]
y_train, y_test = train['target'], test['target']

### 3.2 Add Polynomial Features

In [7]:
for p in range(2,6):
    X_train[f'x{p}'] = X_train['x'] ** p
    X_test[f'x{p}'] = X_test['x'] ** p

In [8]:
X_train.head()

,x,x2,x3,x4,x5
0,2.391265,5.718149,13.673611,32.697231,78.187750
1,4.214828,17.764775,74.875471,315.587233,1330.145907
2,2.801953,7.850938,21.997957,61.637233,172.704606
3,2.448590,5.995595,14.680758,35.947165,88.019886
4,4.671812,21.825832,101.966195,476.366944,2225.497044


In [9]:
X_test.head()

,x,x2,x3,x4,x5
0,1.143986,1.308704,1.497139,1.712706,1.959312
1,5.597518,31.332212,175.382635,981.707524,5495.125928
2,2.213371,4.899010,10.843326,24.000302,53.121567
3,2.269057,5.148620,11.682512,26.508284,60.148808
4,2.974509,8.847701,26.317561,78.281810,232.849908


### 3.3 Train the Model with various data sizes

In [10]:
data_sizes = [10, 25, 50, 100, 200]
train_MSE = []
test_MSE = []
for train_size in data_sizes:
    X = X_train.iloc[:train_size, :]
    y = y_train[:train_size]
    model = LinearRegression(X, y)
    model.fit()
    y_pred_train = model.predict(X).reshape((-1,))
    train_MSE.append(mean_squared_error(y,y_pred_train))
    y_pred_test = model.predict(X_test).reshape((-1,))
    test_MSE.append(mean_squared_error(y_test,y_pred_test))


### 3.4 Plot the Results

In [13]:
fig = px.line(x=data_sizes, y=train_MSE).update_layout(xaxis_title='Number of training samples' , yaxis_title="MSE of training set")
fig.show()

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
در سلول زیر مشابه نمودار فوق را برای دادگان آزمایشی رسم می‌کنیم:
</font>
</p>

In [14]:
fig = px.line(x=data_sizes,y=test_MSE).update_layout(xaxis_title='Number of training samples' , yaxis_title="MSE of test set")
fig.show()

In [15]:
fig = px.line(x=data_sizes,y=test_MSE, range_y=[-5,20]).update_layout(xaxis_title='Number of training samples' , yaxis_title="MSE of test set")
fig.show()

## 4. Analysis

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
همانطور که مشاهده می‌کنید وقتی تعداد دادگان آموزشی ما کم است مدل به خوبی روی دادگان آموزشی برازش می‌شود و درنتیجه خطای آن روی دادگان آموزشی بسیار کم است ولی از طرفی خطای آن روی دادگان تست بسیار زیاد است بنابراین وقتی تعداد دادگان کم است مدل 
<code>overfit</code>
می‌کند.یکی از راه های جلوگیری از
<code>overfitting</code>
افزایش داده های آموزشی است با افزایش داده ها خطای مدل روی داده های آموزشی بیشتر می‌شود ولی خطای دادگان آزمایشی کمتر شده و مدل بهتر
<code>generalize</code>
می‌شود.
</font>
</p>

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
مدلی که با 
<code>10</code>
داده آموزشی برازش یافته است از همه بدتر است زیرا دچار 
<code>overfitting</code>
می‌شود.دو مدلی که با 
<code>100</code>
و
<code>200</code>
داده آموزشی آموزش دیده اند به همدیگر خیلی نزدیک اند زیرا خطای این دو مدل چه روی دادگان آموزشی و چه روی دادگان تست اختلاف خیلی کمی دارند.به نظر می‌رسد افزودن صد داده دیگر کارایی مدل را بیشتر نکرده است.
دو مدل دیگر یعنی مدل هایی که 
<code>50</code>
و 
<code>25</code>
داده آموزشی داشتند عملکرد ضعیف تری نسبت به دو مدل با 
<code>200</code>
و
<code>100</code>
داده آموزشی دارند
اگر مدل ها را با تعداد داده های آموزشی شان به صورت اندیس تشان دهیم رتبه بندی زیر را بین پنج مدل فوق داریم:
</font>
</p>

$$
m_{10} < m_{25} < m_{50} < m_{100} \approx m_{200}
$$

## 5. Linear Regression with Regularization

### 5.1 Read Data

In [47]:
train = pd.read_csv('Dataset/Problem 2/train_set.csv')
test = pd.read_csv('Dataset/Problem 2/test_set.csv')

In [48]:
train.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,target
0,0.11460,20.0,6.96,0.0,0.464,6.538,58.7,3.9175,3.0,223.0,18.6,394.96,7.73,24.4
1,9.39063,0.0,18.10,0.0,0.740,5.627,93.9,1.8172,24.0,666.0,20.2,396.90,22.88,12.8
2,2.37857,0.0,18.10,0.0,0.583,5.871,41.9,3.7240,24.0,666.0,20.2,370.73,13.34,20.6
3,0.05302,0.0,3.41,0.0,0.489,7.079,63.1,3.4145,2.0,270.0,17.8,396.06,5.70,28.7
4,0.14150,0.0,6.91,0.0,0.448,6.169,6.6,5.7209,3.0,233.0,17.9,383.37,5.81,25.3


### 5.2 Scale Data

In [49]:
class Scaler:
    
    def __init__(self):
        self.mean = None
        self.std = None
      
    
    def set_mean(self, X):
        self.mean = np.mean(X, axis=0)
        
    def set_std(self, X):
        self.std = np.std(X, axis=0)
        
    def transform(self, X):
        return (X - self.mean)/self.std
    
    def fit(self, X):
        self.set_mean(X)
        self.set_std(X)
        

In [50]:
X_train = train.drop('target', axis=1)
y_train = train['target']

In [51]:
sc = Scaler()
sc.fit(X_train)
X_train = sc.transform(X_train)
X_train

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13
0,-0.399550,0.395154,-0.617818,-0.282466,-0.790261,0.357484,-0.378985,0.063415,-0.749382,-1.085504,0.029377,0.407320,-0.705939
1,0.614731,-0.473458,1.014199,-0.282466,1.598047,-0.918400,0.889019,-0.925517,1.659831,1.541751,0.789657,0.429288,1.427959
2,-0.151998,-0.473458,1.014199,-0.282466,0.239481,-0.576670,-0.984169,-0.027695,1.659831,1.541751,0.789657,0.132943,0.084237
3,-0.406284,-0.473458,-1.137895,-0.282466,-0.573928,1.115171,-0.220485,-0.173424,-0.864107,-0.806766,-0.350764,0.419776,-0.991868
4,-0.396609,-0.473458,-0.625143,-0.282466,-0.928713,-0.159312,-2.255774,0.912550,-0.749382,-1.026198,-0.303246,0.276077,-0.976374
...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,-0.405469,-0.473458,-1.277071,-0.282466,-0.582582,-0.181721,-0.015154,-0.236895,-0.749382,-1.263422,-0.350764,0.318428,0.057475
402,-0.272836,-0.473458,1.231020,3.540245,0.429853,-0.045869,0.842189,-0.934369,-0.519933,-0.017996,-1.823807,-0.227269,-1.020038
403,1.426195,-0.473458,1.014199,-0.282466,1.251916,-1.408585,1.040314,-1.109667,1.659831,1.541751,0.789657,0.429288,2.544910
404,-0.252071,-0.473458,1.231020,-0.282466,0.429853,1.689388,0.777348,-0.853147,-0.519933,-0.017996,-1.823807,0.174841,-1.551048


### 5.3 Normal Equation 

$$
w = (X^TX + \lambda I)^{-1}X^TY
$$

### 5.4 Implementation

In [52]:
class LinearRegression:
    @staticmethod
    def add_bias(X):
        bias = np.ones((X.shape[0],1))
        #X = X.values
        X = np.concatenate([bias, X], axis=1)
        return X
    
    @staticmethod
    def mse(y_true, y_pred):
        return np.mean((y_true - y_pred)**2)
    
    def pre_process(self):
        self.X = self.add_bias(self.X)
        #y = self.y.values
        self.y = self.y.reshape((-1, 1))
    
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.pre_process()
        self.w = np.zeros((1,X.shape[1]))
        
        
    def predict(self, X):
        X = self.add_bias(X)
        y = np.dot(X,self.w)
        return y
    
    def evaluate(self, X, y=None):
        if y is None:
            y = self.y
        y_pred = self.predict(X)
        return self.mse(y, y_pred)
        
    
    def fit(self, landa):
        X, X_t = self.X, self.X.T
        X_X_t = np.dot(X_t,X)
        reg_mat = landa * np.eye(X_X_t.shape[0])
        psudo_inv = np.linalg.inv(X_X_t + reg_mat)
        self.w = np.dot(np.dot(psudo_inv,X_t),self.y)

### 5.5 Polynomial Feature Extraction

In [53]:
class PolynomialFeatures:
    
    def __init__(self, X, degree):
        self.X = X
        self.degree = degree
        self.row_features = []
        
    def get_row_features(self, row, degree,iter=1, mul=1, idx=0):
        if iter == degree + 1:
            self.row_features.append(mul)
            return
        for i in range(idx, len(row)):
            mul *= row[i]
            self.get_row_features(row, degree, iter+1, mul, i)
            mul /= row[i]
    
    def get_poly_features_row(self, row):
        features = []
        for deg in range(1, self.degree+1):
            self.row_features = []
            self.get_row_features(row, deg)
            features.extend(self.row_features)
        return np.array(features).reshape((1,-1))
    
    def extract_features(self):
        data = self.get_poly_features_row(self.X.iloc[0, :])
        for row_ind in range(1,len(self.X)):
            curr_row = self.get_poly_features_row(self.X.iloc[row_ind])
            data = np.concatenate([data, curr_row], axis=0)
        return data
            

In [54]:
pf = PolynomialFeatures(X_train,3)
X_train = pf.extract_features()
X_train.shape

(406, 559)

### 5.6 Cross validation

In [55]:
def kfold(k, data):
    length = len(data) // k
    folds = []
    for fold_num in range(k):
        row_start, row_end = fold_num * length , (fold_num+1) * length
        if fold_num == k - 1:
            row_end += len(data)%k
        fold_data = data[row_start:row_end, :]
        X_fold, y_fold = fold_data[: , :-1], fold_data[:, -1]
        folds.append((X_fold, y_fold))
    return folds

In [56]:
def get_train_from_folds(folds, idx):
    folds.pop(idx)
    X_folds = [fold[0] for fold in folds]
    y_folds = [fold[1] for fold in folds]
    X_train = np.concatenate(X_folds, axis=0)
    y_train = np.concatenate(y_folds, axis=0)
    return X_train, y_train

def cross_validation(k, model, data, params, repeat=10):
    train_history, valid_history = [], []
    for r in range(repeat):
        np.random.shuffle(data)
        folds = kfold(k, data)
        train_losses, valid_losses = [], []
        for valid_ind, (X_valid, y_valid) in enumerate(folds):
            X_train, y_train = get_train_from_folds(deepcopy(folds), valid_ind)
            m = model(X_train, y_train)
            m.fit(**params)
            train_loss = m.evaluate(X_train)
            valid_loss = m.evaluate(X_valid, y_valid)
            train_losses.append(train_loss)
            valid_losses.append(valid_loss)
        train_history.extend(train_losses)
        valid_history.extend(valid_losses)
    return train_history, valid_history
            

In [57]:
def searchCV(k, model, data, params):
    train_dict, valid_dict = {}, {}
    for lamda_val in params['landa']:
        curr_params = {'landa':lamda_val}
        train_his, valid_his = cross_validation(k, model, data, curr_params)
        train_dict[f'landa:{lamda_val}'] = train_his
        valid_dict[f'landa:{lamda_val}'] = valid_his
    return train_dict, valid_dict     

In [58]:
np.random.seed(42)
y_train = y_train.values.reshape((-1,1))
train_data = np.concatenate([X_train, y_train] , axis=1)
params = {
    'landa':[0.01, 0.03, 0.1, 0.3, 0.5, 0.8, 1]
}
train_cv, valid_cv = searchCV(5, LinearRegression, train_data, params)

In [59]:
px.box(pd.DataFrame(train_cv), title='Cost function of LinearRegression for various hyperparameters in train set')

In [61]:
px.box(pd.DataFrame(valid_cv), log_y=True ,title='Cost function of LinearRegression for various hyperparameters in validation set')